In [ ]:
# @title
# pip

!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q requests bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0 openai

In [ ]:
# @title
# imports

import os
import json
import requests
import gradio as gr
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [ ]:
# @title
# constants

LLAMA = "meta-llama/Llama-3.1-8B-Instruct"
DRIVE_PATH = '/content/drive/MyDrive/'
drive.mount('/content/drive/', force_remount=True)

In [ ]:
# @title
# sign in to HuggingFace

login(userdata.get('HF_TOKEN'), add_to_git_credential=True)

In [ ]:
# @title
# sign in to openAI

openai = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
# @title
# prompts

system_prompt = "You are a helpful AI assistant, Your core mission is to produce synthetic high-quality, diverse datasets tailored precisely to a user's needs."
system_prompt += "For Example: 'I need 25 customer support chat dialogues about internet connectivity issues.' Each dialogue should be between a customer and a support agent, and include at least 3-5 turns per participant."
system_prompt += "The output should be a JSON array."
user_prompt = "I need 10 customer reviews on a vaccume product."

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [ ]:
# @title
# quantization

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
# @title
# tokenize

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
# @title
# build model

inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto")
# model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)

outputs = model.generate(inputs, streamer=streamer, max_new_tokens=2000)

In [ ]:
# @title
# print response

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# @title
# write output to google drive

response_folder = os.path.join(DRIVE_PATH, "04_Projects", "Python Projects",  "synthetic_data_generator", "Llama_Responses")
if not os.path.exists(response_folder):
    os.makedirs(response_folder)
    print(f"Created folder: {response_folder}")
else:
    print(f"Using existing folder: {response_folder}")

def save_response_to_drive(file_content, filename):
    """
    Saves a string to a file in the mounted Google Drive.
    """
    try:
        # Construct the full file path within your mounted Drive
        full_path = os.path.join(response_folder, filename)

        with open(full_path, "w") as f:
            f.write(file_content)

        print(f"File '{filename}' saved successfully to Google Drive at: {full_path}")
        return True
    except Exception as e:
        print(f"Error saving file to Google Drive: {e}")
        return False

In [ ]:
# @title
save_response_to_drive(response, "vacume_reviews.txt")

In [ ]:
# @title
# build combined function

def generate(prompt):
  messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt}
  ]
  inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
  outputs = model.generate(inputs, streamer=streamer, max_new_tokens=5000)
  response = tokenizer.decode(outputs[0], skip_special_tokens=True)
  yield response

In [ ]:
# @title
# build UI

ui = gr.Interface(
    fn=generate,
    inputs=["text"],
    outputs=["text"],
    allow_flagging="never"
)

ui.launch()

In [ ]:
# @title

